In [ ]:
import tensorflow as tf
import tensorflow.keras.layers as layers
import tensorflow.keras.utils as utils
import tensorflow.keras.models as models
import tensorflow.keras.callbacks as callbacks
import tensorflow.keras.preprocessing.image as image
import tensorflow.keras.optimizers as optimizers
import pandas as pd
import numpy as np
import cv2
import matplotlib.pyplot as plt
import math
from imgaug import augmenters as iaa
from sklearn.model_selection import train_test_split
from PIL import Image

In [ ]:
train_labels = pd.read_csv('train_labels.csv',dtype= {'Id': 'str', 'Category': 'str'})

In [ ]:
IMAGE_SIZE = 224

In [ ]:
X = []
y = []
for i in range(train_labels.shape[0]):
    img = image.load_img('train/{}.jpg'.format(train_labels.iloc[i]['Id']),
                                                target_size=(IMAGE_SIZE, IMAGE_SIZE))
    X.append(image.img_to_array(img)/255.0)
    y.append(train_labels.iloc[i]['Category'])

In [ ]:
X = np.asarray(X)

In [ ]:
from tensorflow.keras.utils import to_categorical
classes = {'rose': 0, 'sunflower': 1, 'daisy': 2, 'tulip': 3, 'dandelion': 4}
yi = [classes[s] for s in y]
y_sparse = to_categorical(yi)

In [ ]:
train_X, val_X, train_Y, val_Y = train_test_split(X, y_sparse, test_size=0.1, random_state=17, stratify=y_sparse)

In [ ]:
base_model = tf.keras.applications.NASNetLarge(weights='imagenet', include_top=False)

In [ ]:
x = base_model.output
x = layers.SpatialDropout2D(0.2)(x)
x = layers.GlobalAvgPool2D()(x)
x = layers.Dense(4096, activation='relu')(x)
x = layers.Dropout(0.2)(x)
predictions = layers.Dense(len(classes), activation='softmax')(x)

m = models.Model(inputs=base_model.input, outputs=predictions)

In [ ]:
class AugmentedSequence(utils.Sequence):
  def __init__(self, X, y, batch_size):
    self.X = np.array(X)
    self.y = np.array(y)
    self.batch_size = batch_size
    
    #for shuffling
    self.ids = np.random.permutation(range(len(X)))
    
    #for augmentation
    self.seq = iaa.Sequential([
        iaa.Fliplr(0.5),
        iaa.Flipud(0.2),
        iaa.Affine(scale=(0.8, 1.2), translate_percent=(-0.2, 0.2), rotate=(-45, 45), shear=(-16, 16))
    ])
    
  def __len__(self):
    return int(np.ceil(len(self.X)/float(self.batch_size)))
  
  def __getitem__(self, index):
    start = index * self.batch_size
    end = np.minimum((index + 1) * self.batch_size, len(self.X))
    
    ids = self.ids[start:end]
    
    batchX = self.X[ids]
    batchy = self.y[ids]
    
    batchX = self.seq.augment_images(batchX)
    
    return np.array(batchX), np.array(batchy)
  
  def on_epoch_end(self):    
    self.ids = np.random.permutation(range(len(self.X)))

In [ ]:
base_model.trainable = False
m.compile(optimizer=optimizers.Adam(1e-6),
          loss='categorical_crossentropy',
          metrics=['accuracy'])
m.fit_generator(AugmentedSequence(train_X, train_Y, 8), 
      validation_data=(val_X, val_Y),
      epochs = 20)

In [ ]:
base_model.trainable = True
m.compile(optimizer=optimizers.Adam(1e-6),
          loss='categorical_crossentropy',
          metrics=['accuracy'])
m.fit_generator(AugmentedSequence(train_X, train_Y, 8), 
      validation_data=(val_X, val_Y),
      epochs = 100,
      callbacks=[
          callbacks.ModelCheckpoint('weights-progress.h5', verbose=1, save_best_only=True, save_weights_only=True)
      ])

In [ ]:
base_model.trainable = True
m.compile(optimizer=optimizers.Adam(1e-8),
          loss='categorical_crossentropy',
          metrics=['accuracy'])
m.fit_generator(AugmentedSequence(np.concatenate((train_X, val_X)), np.concatenate((train_Y, val_Y)), 8), 
      validation_data=(val_X, val_Y),
      epochs = 20
)

In [ ]:
sub_num=5

In [ ]:
m.save_weights(f'weights-submission-{sub_num}.h5')

In [ ]:
import os
test_X = []
test_lebels = []
for img_file in os.listdir('test'):
    img = image.load_img('test/' + img_file, target_size=(IMAGE_SIZE, IMAGE_SIZE))
    test_X.append(image.img_to_array(img)/255.0)
    test_lebels.append(img_file.replace('.jpg', ''))
test_X = np.asarray(test_X)

In [ ]:
test_pred = m.predict(test_X)

In [ ]:
class_indices = tf.argmax(test_pred, axis=1).eval(session=tf.Session())
reverted_classes = {classes[c]:c for c in classes}
pred_classes = [reverted_classes[c] for c in class_indices]

In [ ]:
df = pd.DataFrame(data={'Category': pred_classes}, index=test_lebels)

In [ ]:
df.index.name = 'Id'
df.sort_index().to_csv(f'res-{sub_num}.csv')